## Imports

In [198]:
!pip install pandas-profiling
!pip install geojson
!pip install shapely

import pandas as pd
import folium
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import pandas_profiling
import datetime
import geojson
from shapely.geometry import shape, Point
import json

### BOKEH
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import (
    ColumnDataSource, RangeTool, Legend, Label, LogColorMapper, ColorBar,  GeoJSONDataSource,
    HoverTool, LinearColorMapper)
from bokeh.plotting import figure
from bokeh.palettes import Viridis6, Viridis11, RdYlBu11
from bokeh.io import show, output_notebook, output_file, curdoc
from bokeh.themes import built_in_themes

### PLOTLY
import plotly.express as px


### Change from baseline 
Create mobility dataset using [Google Mobility Report](https://www.google.com/covid19/mobility/) of 2020 and 2021

In [241]:
# Upload Mobility report dataset for 2021
df = pd.read_csv('datasets/2021_GB_Region_Mobility_Report.csv')

# Upload Mobility report dataset for 2021
df0 = pd.read_csv('datasets/2020_GB_Region_Mobility_Report.csv')

# Concat datasets
df = pd.concat([df0, df])
df.drop(['country_region_code', 'country_region'], axis=1)

# Upload restrictions dataset
df1 = pd.read_csv('datasets/restrictions_daily.csv')

# Merge datatsets
data = pd.merge(df, df1, how="inner", on=["date"])

# Obtain report of the dataset
# prof = pandas_profiling.ProfileReport(data)
# prof.to_file(output_file='output.html')

data.to_csv('datasets/london_mobility_restrictions.csv')

In [242]:
# Upload Mobility report datasets

# Upload Mobility report dataset for 2021
df = pd.read_csv('datasets/2021_GB_Region_Mobility_Report.csv')

# Upload Mobility report dataset for 2021
df0 = pd.read_csv('datasets/2020_GB_Region_Mobility_Report.csv')

# Concat datasets
df = pd.concat([df0, df])


# Extract Greater London Area information
df.drop(['country_region_code', 'country_region'], axis=1)
df = df.loc[df.sub_region_1 == 'Greater London']
df.sub_region_2 = df.sub_region_2.fillna(0)
df = df.loc[df.sub_region_2 == 0]
df.date = pd.to_datetime(df.date)


# Bokeh
curdoc().theme = 'night_sky'

p = figure(plot_height=300, plot_width=800, tools=["xpan"], toolbar_location=None, title="Mobility in Greater London",
           x_axis_type="datetime", x_range=(min(df.date), max(df.date)))

line ={}
legend_items = []


for index, baseline_col in enumerate(list(df.columns.values)[-6:]):
    
    line[baseline_col] = p.line(df.date, df[baseline_col], color=Viridis6[index],
                    alpha=1, muted_color=Viridis6[index], muted_alpha=0.1, line_width=2)
    
    # Add legend and make it readable
    legend_items.append((baseline_col.replace('_', ' ')[:-28], [line[baseline_col]]))


legend = Legend(items=legend_items)
p.add_layout(legend,'left')
p.legend.click_policy = 'mute'


# Add baseline line
p.line(df.date, 0, color='black', line_width=0.5, line_dash='dashed')


p.yaxis.axis_label = 'Change from baseline'

select = figure(title="Drag the middle and edges of the selection box to change the date range above",
                plot_height=100, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None)

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2



for baseline_col in list(df.columns.values)[-6:]:
    select.line(df.date, df[baseline_col])
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
select.background_fill_alpha = 0.0
select.border_fill_alpha = 0.0
legend.background_fill_alpha = 0.0
legend.border_line_alpha = 0.0


show(column(p, select))


output_file("data/ChangeBaseline.html")


### Restrictions timeline

In [243]:
# Create dataset using the restrictions dataset
df = pd.DataFrame([
    dict(Task='schools_closed_0', Start='2020-03-23', Finish='2020-05-31', Resource='schools closed'),
    dict(Task='pubs_closed_0',  Start='2020-03-21',Finish='2020-07-02', Resource='pubs closed'),
    dict(Task='shops_closed_0',  Start='2020-03-24',Finish='2020-06-14', Resource='shops closed'),
    dict(Task='eating_places_closed_0',  Start='2020-03-21',Finish='2020-07-03', Resource='eating places closed'),
    dict(Task='schools_closed_1',  Start='2021-01-05',Finish='2021-03-07', Resource='schools closed'),
    dict(Task='pubs_closed_1',  Start='2020-11-05',Finish='2020-12-01', Resource='pubs closed'),
    dict(Task='shops_closed_1',  Start='2020-11-05',Finish='2020-12-01', Resource='shops closed'),
    dict(Task='eating_places_closed_1',  Start='2020-11-05',Finish='2020-12-01', Resource='eating places closed'),
    dict(Task='pubs_closed_2',  Start='2020-12-16',Finish='2021-04-18', Resource='pubs closed'),
    dict(Task='shops_closed_2',  Start='2020-12-20',Finish='2021-04-18', Resource='shops closed'),
    dict(Task='eating_places_closed_2',  Start='2020-12-16', Finish='2021-04-18', Resource='eating places closed'),
    dict(Task='stay_at_home_0',  Start='2020-03-24', Finish='2020-05-10', Resource='stay at home'),
    dict(Task='stay_at_home_1',  Start='2020-11-05',Finish='2020-12-02', Resource='stay at home'),
    dict(Task='stay_at_home_2',  Start='2020-12-20', Finish='2021-03-28', Resource='stay at home'),
    dict(Task='household_mixing_indoors_banned_0',  Start='2020-03-24', Finish='2020-07-03', Resource='household mixing indoors banned'),
    dict(Task='household_mixing_indoors_banned_1',  Start='2020-10-17', Finish='2021-04-18', Resource='household mixing indoors banned'),
    dict(Task='wfh_0',  Start='2020-03-17', Finish='2020-07-31', Resource='wfh'),
    dict(Task='wfh_1',  Start='2020-09-22', Finish='2021-04-18', Resource='wfh'),
    dict(Task='rule_of_6_indoors_0',  Start='2020-09-14', Finish='2020-10-16', Resource='rule of 6 indoors'),
    dict(Task='curfew_0',  Start='2020-09-24', Finish='2020-11-04', Resource='curfew'),
    dict(Task='curfew_1',  Start='2020-12-02', Finish='2020-12-15', Resource='curfew'),
    dict(Task='eat_out_to_help_out_0',  Start='2020-08-03',Finish='2020-08-30', Resource='eat out to help out'),
])

# Plot timeline of restrictions using plotly
fig = px.timeline(df, x_start="Start", x_end="Finish",
                  y="Resource", color="Resource", labels={
                     "Start": "Start",
                     "Finish": "Finish",
                     "Resource": "Restriction"
                 },
                title="Timeline of mobility restrictions", template='plotly_dark')
fig.update_xaxes(
    tickmode = 'auto',
    nticks =  20,
    tickformat="%b %y")


fig.update_layout(showlegend=False, plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)' )


fig.show()
fig.write_html("data/timeline_restrictions.html")

### Mobility infrastructure of london

In [244]:
# Obtain number of bus stops per borough, number of docking stations per borough, number of tfl stops per borough
    
with open('datasets/london_boroughs.geojson') as f:
    london = geojson.load(f)
    
with open('datasets/stops_with_lat_longs.geojson') as f:
    stops = geojson.load(f)
    
with open('datasets/cycle_hire_locations.geojson') as f:
    stations = geojson.load(f)   

with open('datasets/tfl.geojson') as f:
    tfl = geojson.load(f)   
    
        
# Transform coordenates to Points
bus_points = []
for i in range(len(stops['features'])):
    bus_points.append(Point(stops['features'][i]['geometry']['coordinates']
                  [0], stops['features'][i]['geometry']['coordinates'][1]))

docking_points = []
for i in range(len(stations['features'])):
    docking_points.append(Point(stations['features'][i]['properties']['longitude'],
                   stations['features'][i]['properties']['latitude']))
    
tfl_points = []
for i in range(len(tfl['features'])):
     tfl_points.append(Point(tfl['features'][i]['geometry']['coordinates']
                  [0], tfl['features'][i]['geometry']['coordinates'][1]))

# Check if borough contains each point        
for feature in london['features']:
    polygon = shape(feature['geometry'])
    feature['properties']['stops_counter']  = 0
    feature['properties']['docking_counter']  = 0
    feature['properties']['tfl_counter']  = 0
    
    # Count stops
    for point in bus_points:
        if polygon.contains(point):
            feature['properties']['stops_counter'] +=1
            
    # Count docking stations       
    for point in docking_points:
        if polygon.contains(point):
            feature['properties']['docking_counter'] +=1
            
    # Count docking stations       
    for point in tfl_points:
        if polygon.contains(point):
            feature['properties']['tfl_counter'] +=1
            
with open('datasets/london_stops.geojson', 'w') as f:
    json.dump(london, f)            

#### Bus

In [245]:
# Plot Bus Stops per Borough

with open('datasets/london_stops.geojson') as f:
    london_source = GeoJSONDataSource(geojson=f.read())


# Define color mapper    
color_mapper = LinearColorMapper(palette=Viridis11, low=0, high=1200)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

curdoc().theme = 'night_sky'

p = figure(title="Number of bus stops per borough", tools=TOOLS, x_axis_location=None,
           y_axis_location=None, width=600, height=400)
p.grid.grid_line_color = None


# Plot map of Greater London
p.patches('xs', 'ys', source=london_source,
          fill_color={'field': 'stops_counter', 'transform': color_mapper},
          fill_alpha=1.0, line_color="black", line_width=0.5)

# Add legend
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))

p.add_layout(color_bar, 'right')

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
color_bar.background_fill_alpha = 0.0


hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@name"), ("Number of stops", "@stops_counter")]

output_file("data/London_bus_stops.html", title="Number of bus stops per borough")

show(p)

#### Cycle

In [246]:
# Plot cycle hire docking stations per Borough

with open('datasets/london_stops.geojson') as f:
    london_source = GeoJSONDataSource(geojson=f.read())

# Define color mapper       
color_mapper = LinearColorMapper(palette=Viridis11, low=0, high=200)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

curdoc().theme = 'night_sky'

p = figure(title="Number of Santander Cycle Hire docking stations per borough", tools=TOOLS, x_axis_location=None,
           y_axis_location=None, width=600, height=400)
p.grid.grid_line_color = None


# Plot map of Greater London
p.patches('xs', 'ys', source=london_source,
          fill_color={'field': 'docking_counter', 'transform': color_mapper},
          fill_alpha=1.0, line_color="black", line_width=0.5)


color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))

p.add_layout(color_bar, 'right')

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
color_bar.background_fill_alpha = 0.0

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@name"), ("Number of docking stations", "@docking_counter")]

output_file("data/London_docking_stations.html", title="Number of bus stops per borough")

show(p)

In [247]:
# Plot number of cycle hires

cycle_hires = pd.read_excel('datasets/tfl-daily-cycle-hires.xlsx', sheet_name='Data')


cycle_hires = cycle_hires.loc[cycle_hires['Day']>'2017-01-01']

curdoc().theme = 'night_sky'

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_range=(datetime.datetime(2017,1,1), datetime.datetime(2021, 4, 1)))

p.circle(cycle_hires['Day'], cycle_hires['Number of Bicycle Hires'])

p.yaxis.axis_label = 'Bicycle Hires'





select = figure(title="Drag the middle and edges of the selection box to change the date range above",
                plot_height=100, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None)



p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
select.background_fill_alpha = 0.0
select.border_fill_alpha = 0.0


range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line(cycle_hires['Day'], cycle_hires['Number of Bicycle Hires'])
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))

output_file("data/CycleHires.html")

In [257]:
# Plot cycle routes
with open('datasets/london_boroughs.geojson') as f:
    london_source = GeoJSONDataSource(geojson=f.read())

with open('datasets/CycleRoutes.geojson') as f:
    cycle_source = GeoJSONDataSource(geojson=f.read())

    
with open('datasets/cycle_hire_locations.geojson') as f:
    docking_source = GeoJSONDataSource(geojson=f.read())


TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="Cycle Routes", tools=TOOLS, x_axis_location=None,
           y_axis_location=None, width=600, height=400)
p.grid.grid_line_color = None


# Plot map of Greater London
p.patches('xs', 'ys', source=london_source,
          fill_color=Viridis6[0],
          fill_alpha=0.5, line_color="black", line_width=0.5)

# Plot cycle routes on top
p.multi_line('xs', 'ys', source=cycle_source, line_alpha=1, line_color=Viridis6[5],
             line_width=0.7)

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
        
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@name")]

output_file("data/Cycle_Routes.html", title="Cycle Routes")

show(p)

#### TLF STATIONS (Tube, DLR, Overground, Tramlink, Emirates Air Line and Crossrail)

In [248]:
# Plot TFL stations per Borough

with open('datasets/london_stops.geojson') as f:
    london_source = GeoJSONDataSource(geojson=f.read())

# Define color mapper 
color_mapper = LinearColorMapper(palette=Viridis11, low=0, high=40)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

curdoc().theme = 'night_sky'

p = figure(title="Number of TFL Stations per borough", tools=TOOLS, x_axis_location=None,
           y_axis_location=None, width=600, height=400)
p.grid.grid_line_color = None


# Plot map of Greater London
p.patches('xs', 'ys', source=london_source,
          fill_color={'field': 'tfl_counter', 'transform': color_mapper},
          fill_alpha=1.0, line_color="black", line_width=0.5)


color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))

p.add_layout(color_bar, 'right')

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
color_bar.background_fill_alpha = 0.0

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@name"), ("Number of TFL stations", "@tfl_counter")]

output_file("data/London_tfl_stations.html", title="Number of TFL Stations per borough")

show(p)

In [260]:
# todo: plot or remove

from matplotlib.dates import DateFormatter

df2 = pd.read_excel('datasets/tfl-journeys-type.xlsx', sheet_name=1)

df2 = df2.loc[df2['Period beginning'] > '2020-01-01']

df2

fig  = px.bar(df2['Period beginning'], df2['Bus journeys (m)'], 
                title="Timeline of mobility restrictions", template='plotly_dark', )



fig.update_layout(showlegend=False, plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)' )


fig.show()
# fig.write_html("data/timeline_restrictions.html")

'''Problem: this dataset has the MONTHLY JOURNEYS'''

# df = df.loc[df['Period beginning'] > '2020-01-01']
# plt.figure(figsize=(20, 10))
# # plt.plot(df['Period beginning'], df['Bus journeys (m)'], '.-r', label='Bus')
# # plt.plot(df['Period beginning'], df['Underground journeys (m)'], '.-y', label='Underground')
# # plt.plot(df['Period beginning'], df['DLR Journeys (m)'], '.-g', label='DLR')
# # plt.plot(df['Period beginning'], df['Tram Journeys (m)'], '.-b', label='Tram')
# legend= ['Bus', 'Underground','DLR', 'Tram' ]
# plt.stackplot(df['Period beginning'],df['Bus journeys (m)'], df['Underground journeys (m)'],df['DLR Journeys (m)'], df['Tram Journeys (m)'] , labels=legend )
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.title('Number of journeys on the public transport network by type of transport (in millions)', fontsize=20)
# plt.show()

'Problem: this dataset has the MONTHLY JOURNEYS'

Maps made with folium - Google Mobility Data

In [ ]:
mobility = pd.read_csv('data\cleaned_sub_region_2_data.csv')

In [ ]:
mobility_mean_first = pd.DataFrame(columns=['name', 'retail_and_recreation_percent_change_from_baseline_first', 'grocery_and_pharmacy_percent_change_from_baseline_first', 'parks_percent_change_from_baseline_first', 'transit_stations_percent_change_from_baseline_first', 'workplaces_percent_change_from_baseline_first', 'residential_percent_change_from_baseline_first'])

In [ ]:
for index, borough in enumerate(mobility.sub_region_2.unique()):
    mask = ('2020-03-24' <= mobility['date']) & (mobility['date'] <= '2020-03-30' ) & (mobility['sub_region_2'] == borough)
    borough_df = mobility.loc[mask]
    mobility_mean_first.loc[index] = [
        borough, 
        borough_df.retail_and_recreation_percent_change_from_baseline.mean(), 
        borough_df.grocery_and_pharmacy_percent_change_from_baseline.mean(), 
        borough_df.parks_percent_change_from_baseline.mean(), 
        borough_df.transit_stations_percent_change_from_baseline.mean(), 
        borough_df.workplaces_percent_change_from_baseline.mean(), 
        borough_df.residential_percent_change_from_baseline.mean()
        ]

In [ ]:
mobility_mean_first

In [ ]:
mobility_mean_second = pd.DataFrame(columns=['name', 'retail_and_recreation_percent_change_from_baseline_second', 'grocery_and_pharmacy_percent_change_from_baseline_second', 'parks_percent_change_from_baseline_second', 'transit_stations_percent_change_from_baseline_second', 'workplaces_percent_change_from_baseline_second', 'residential_percent_change_from_baseline_second'])

In [ ]:
for index, borough in enumerate(mobility.sub_region_2.unique()):
    mask = ('2020-11-05' <= mobility['date']) & (mobility['date'] <= '2020-11-11' ) & (mobility['sub_region_2'] == borough)
    borough_df = mobility.loc[mask]
    mobility_mean_second.loc[index] = [
        borough, 
        borough_df.retail_and_recreation_percent_change_from_baseline.mean(),
        borough_df.grocery_and_pharmacy_percent_change_from_baseline.mean(), 
        borough_df.parks_percent_change_from_baseline.mean(), 
        borough_df.transit_stations_percent_change_from_baseline.mean(), 
        borough_df.workplaces_percent_change_from_baseline.mean(), 
        borough_df.residential_percent_change_from_baseline.mean()]

In [ ]:
mobility_mean_second

In [ ]:
data_final_first = pd.DataFrame(columns=[
    'name',
    'retail_and_recreation_percent_change_from_baseline_first',
    'grocery_and_pharmacy_percent_change_from_baseline_first',
    'parks_percent_change_from_baseline_first',
    'transit_stations_percent_change_from_baseline_first',
    'workplaces_percent_change_from_baseline_first',
    'residential_percent_change_from_baseline_first'
])

In [ ]:
data_final_second = pd.DataFrame(columns=[
    'name',
    'retail_and_recreation_percent_change_from_baseline_second',
    'grocery_and_pharmacy_percent_change_from_baseline_second',
    'parks_percent_change_from_baseline_second',
    'transit_stations_percent_change_from_baseline_second',
    'workplaces_percent_change_from_baseline_second',
    'residential_percent_change_from_baseline_second'
])

In [ ]:
df_final = data_final_first.join(data_final_second.set_index('name'), on='name')

In [ ]:
london_boroughs = f"data\london_boroughs.geosjon"

In [ ]:
rr1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['retail_and_recreation_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'retail_and_recreation_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Retail and Recreation \% Change from Baseline (First Lockdown)',
                  name="Retail and Recreation \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(rr1)

In [ ]:
gp1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['grocery_and_pharmacy_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'grocery_and_pharmacy_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Grocery and Pharmacy \% Change from Baseline (First Lockdown)',
                  name="Grocery and Pharmacy \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(gp1)

In [ ]:
p1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['parks_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'parks_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Parks \% Change from Baseline (First Lockdown)',
                  name="Parks \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(p1)

In [ ]:
ts1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['transit_stations_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'transit_stations_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Transit Stations \% Change from Baseline (First Lockdown)',
                  name="Transit Stations \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(ts1)

In [ ]:
w1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['workplaces_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'workplaces_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Workplaces \% Change from Baseline (First Lockdown)',
                  name="Workplaces \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(w1)

In [ ]:
rr2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['retail_and_recreation_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'retail_and_recreation_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Retail and Recreation \% Change from Baseline (Second Lockdown)',
                  name="Retail and Recreation \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(rr2)

In [ ]:
gp2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['grocery_and_pharmacy_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'grocery_and_pharmacy_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Grocery and Pharmacy \% Change from Baseline (Second Lockdown)',
                  name="Grocery and Pharmacy \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(gp2)

In [ ]:
p2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['parks_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'parks_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Parks \% Change from Baseline (Second Lockdown)',
                  name="Parks \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(p2)

In [ ]:
ts2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['transit_stations_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'transit_stations_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Transit Stations \% Change from Baseline (Second Lockdown)',
                  name="Transit Stations \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(ts2)

In [ ]:
w2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['workplaces_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'workplaces_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Workplaces \% Change from Baseline (Second Lockdown)',
                  name="Workplaces \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(w2)